In [11]:
import json
import numpy as np
from tqdm import tqdm
from pprint import pprint
from sklearn.cluster import KMeans

from label_processor import CStdLib

In [12]:
with open('result_logits.json', encoding='utf-8') as f:
    logits_json = json.load(f)

In [13]:
embed = CStdLib(single=False)

In [14]:
pbar = tqdm(logits_json.items())
i = 0
for key, prod_dict in pbar:
    optional_tags = prod_dict['optional_tags']
    num_class = len(optional_tags)
    imgs_tags = prod_dict['imgs_tags']
    imgs_logits = []
    for i in range(len(imgs_tags)):
        imgs_logits.append(np.fromstring(imgs_tags[i][key + '_{}.jpg'.format(i)], dtype=np.float32, sep=' '))
    imgs_logits = np.array(imgs_logits)
    
    # 聚类
    cluster = KMeans(n_clusters=num_class, random_state=42).fit(imgs_logits)
    labels = cluster.labels_
    
    # 计算匹配度
    match_score = np.zeros((num_class, num_class))
    # match_score[i, j] 表示 kmeans 分类结果为 i 和 optional_tags[j] 的匹配度
    embeded_tags = [embed(tag) for tag in optional_tags]
    not_matched = False
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        logits = imgs_logits[id_i]
        for j in range(num_class):
            logits_ids = embeded_tags[j]
            if len(logits_ids) > 0:
                logits_j = logits[:, logits_ids]
                match_score[i, j] = np.mean(logits_j)
            else:
                not_matched = True
                match_score[i, j] = 1
    
    # 对 scores 进行归一化，使得每一列的 ord 范数为 1；然后在每一行中找到最大值，即为该行商品对应的标签。
    # 按列归一化，按行找最大值，这样就做到了双向选择。
    # 使用较高阶的范数能增大未匹配标签对应的概率，有利于提高 acc/em。
    match_score_norm = match_score / np.linalg.norm(match_score, axis=0, ord=3)
    # print(key, '\n', match_score_norm)
    label_id = np.argmax(match_score_norm, axis=1)
    label_score = np.max(match_score_norm, axis=1)
    # print(label_id)
    
    # 强制使每一类都被选中
    selected_label_id = []
    for i in range(num_class):
        if label_id[i] == -1:
            continue
        selected_label_id.append(label_id[i])
        for j in range(i + 1, num_class):
            if label_id[i] == label_id[j]:
                if label_score[i] > label_score[j]:
                    label_id[j] = -1
                else:
                    label_id[i] = -1
    selected_label_id = set(selected_label_id)
    unselected = list(set(range(num_class)) - selected_label_id)
    i = 0
    for j in range(num_class):
        if label_id[j] == -1:
            label_id[j] = unselected[i]
            i += 1
    
    # 终于可以写入 json 了
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        for j in id_i:
            logits_json[key]['imgs_tags'][j][key + '_{}.jpg'.format(j)] = optional_tags[label_id[i]]
    if not_matched:
        print(key, '\n', match_score, '\n', match_score_norm)
        pprint(logits_json[key])

  5%|▌         | 280/5331 [00:02<00:50, 100.60it/s]

624952669722 
 [[3.58562636 1.        ]
 [5.74702406 1.        ]] 
 [[0.58029295 0.79370053]
 [0.93009064 0.79370053]]
{'imgs_tags': [{'624952669722_0.jpg': '珊瑚色组合'},
               {'624952669722_1.jpg': '珊瑚色组合'},
               {'624952669722_2.jpg': '橄榄色组合'},
               {'624952669722_3.jpg': '橄榄色组合'},
               {'624952669722_4.jpg': '橄榄色组合'}],
 'optional_tags': ['珊瑚色组合', '橄榄色组合']}


  7%|▋         | 349/5331 [00:03<00:50, 97.89it/s] 

614043736149 
 [[ 5.96085691  5.81517696  2.98464155  1.          6.27514553]
 [ 3.44056153  3.20435333 11.78937626  1.          3.06254959]
 [ 3.48982143 11.48388481  4.44730949  1.          3.26497936]
 [12.58054924  2.30453396  2.99636054  1.          2.12128043]
 [ 3.87575579  2.04849195  1.98173594  1.         13.41447067]] 
 [[0.44871058 0.48119926 0.24591403 0.58480355 0.44876604]
 [0.25899235 0.26515658 0.97136387 0.58480355 0.21901775]
 [0.26270045 0.95027836 0.36642785 0.58480355 0.23349449]
 [0.94701577 0.19069755 0.24687959 0.58480355 0.15170304]
 [0.29175211 0.16951037 0.16328147 0.58480355 0.95933376]]
{'imgs_tags': [{'614043736149_0.jpg': '条纹'},
               {'614043736149_1.jpg': '桔色'},
               {'614043736149_2.jpg': '红色'},
               {'614043736149_3.jpg': '桔色'},
               {'614043736149_4.jpg': '红色'},
               {'614043736149_5.jpg': '黑色'},
               {'614043736149_6.jpg': '条纹'},
               {'614043736149_7.jpg': '黑色'},
               {

 15%|█▌        | 814/5331 [00:07<00:49, 92.17it/s] 

...

 85%|████████▍ | 4526/5331 [00:42<00:06, 116.79it/s]

577541680180 
 [[15.40407181  1.          7.24776173  1.84761405]
 [ 1.82517242  1.          2.26344609 11.71285343]
 [ 3.90824056  1.         12.36336899  3.2937541 ]
 [ 2.89369607  1.          4.93059397 10.03467846]] 
 [[0.99192408 0.62996052 0.54118827 0.13335765]
 [0.11752948 0.62996052 0.16901087 0.84541389]
 [0.25166579 0.62996052 0.92316919 0.23773758]
 [0.18633559 0.62996052 0.36816603 0.72428607]]
{'imgs_tags': [{'577541680180_0.jpg': '红色'},
               {'577541680180_1.jpg': '红色'},
               {'577541680180_2.jpg': '蓝色'},
               {'577541680180_3.jpg': '蓝色'},
               {'577541680180_4.jpg': '橙色'},
               {'577541680180_5.jpg': '深色'},
               {'577541680180_6.jpg': '橙色'},
               {'577541680180_7.jpg': '蓝色'},
               {'577541680180_8.jpg': '蓝色'}],
 'optional_tags': ['橙色', '深色', '红色', '蓝色']}


 90%|████████▉ | 4786/5331 [00:44<00:04, 120.51it/s]

612126042525 
 [[ 1.         13.88354111]
 [ 1.         12.36282253]] 
 [[0.79370053 0.83688756]
 [0.79370053 0.74521999]]
{'imgs_tags': [{'612126042525_0.jpg': '957黑色'},
               {'612126042525_1.jpg': '957黑色'},
               {'612126042525_2.jpg': '957黑色'},
               {'612126042525_3.jpg': '6802格子'},
               {'612126042525_4.jpg': '957黑色'},
               {'612126042525_5.jpg': '957黑色'},
               {'612126042525_6.jpg': '957黑色'}],
 'optional_tags': ['6802格子', '957黑色']}


100%|██████████| 5331/5331 [00:49<00:00, 108.33it/s]


In [15]:
with open('./result_labels.json', 'w') as f:
    json.dump(logits_json, f, indent=4, ensure_ascii=False)